In [345]:
from preparation import *
import copy

In [346]:
R = ['A','B','C','D','E','F']

F = [[['A'],['B']],[['B'],['C','D']],[['D'],['E']],[['C','E'],['F']]]

In [347]:
class NF:
    def __init__(self,R,F):
        self.R = R
        self.F =F

    def check_NontrivialDependencyCaseViolationOfBCNF(self,s,schemaR):
        """
        检查不常用（non-trivial）依赖s是否导致不符合BCNF
        :param s:F中的一个在schemaR上的非平凡依赖;ex:[['A'],['B']]
        schemaR:list,表的属性;ex:['A','B','C','D','E','F']
        :return:boolean,True:符合BC范式；False：违背BC范式
        """
        s_plus_set = set(calculate_single_attribute_closure(s[0],self.F))
        R_set = set(schemaR)
        # print(s_plus_set,'\t',R_set,'\t',R_set.issubset(s_plus_set))
        return   R_set.issubset(s_plus_set)
        # return schemaR.issubset(s_plus_set)
    def check_schemaInBCNF(self,schemaR:list):
        """
        检查schena是否符合BC范式
        :param schemaR: list,表的属性;ex:['A','B','C','D','E','F']
        :return: boolean,True:符合BC范式；False：违背BC范式
        """
        result = True
        vio = None
        for i in self.F:
            alpha_set = set(i[0])
            beta_set = set(i[1])
            if  not beta_set.issubset(alpha_set) and (alpha_set.union(beta_set)).issubset(set(schemaR)):#保证该依赖是非平凡依赖且在Ri中，可以被分解
                if not self.check_NontrivialDependencyCaseViolationOfBCNF(i,schemaR):#检查该依赖是否违背BC范式
                #     result = True
                # else:
                    vio = i
                    result = False
                    break
        return result,vio

    def BCNF(self):
        """
        BC范式分解
        :return: BC范式分解结果
        """
        result = [self.R]
        done = False
        test = 0
        step = {'root':self.R}
        while(not done):
            allBCNF = True
            for i in range(len(result)):
                temp_R = result[i]#list
                temp_R_set = set(temp_R)
                temp_done = False
                # print(temp_R,self.check_schemaInBCNF(temp_R))
                bc,vio = self.check_schemaInBCNF(temp_R)
                if not bc:##
                    # print(allBCNF)
                    allBCNF = False
                    # print(allBCNF)
                    alpha =vio[0]#list
                    beta = vio[1]#list
                    alpha_set = set(alpha)
                    beta_set = set(beta)
                    if alpha_set.isdisjoint(beta_set):#alpha和beta的交集为空
                        step['decompose_dependency']= vio
                        result.remove(temp_R)
                        result.append(list(alpha_set.union(beta_set)))
                        step['left'] = list(alpha_set.union(beta_set))
                        result.append(list(temp_R_set.difference(beta_set)))
                        step['right'] = {'root':list(temp_R_set.difference(beta_set))}
                        # temp_done = True
                    # for s in self.F:
                    #     alpha =s[0]#list
                    #     beta = s[1]#list
                    #     alpha_set = set(alpha)
                    #     beta_set = set(beta)
                    #     # print(temp_R,'\n',alpha,beta,'\t')
                    #     if alpha_set.union(beta_set).issubset(temp_R_set):#alpha->beta 是Ri上的函数依赖
                    #         # print(alpha_set.union(beta_set).issubset(temp_R_set))
                    #         if not beta_set.issubset(alpha_set):#alpha->beta 是非平凡函数依赖
                    #             if alpha_set.isdisjoint(beta_set):#alpha和beta的交集为空
                    #                 temp = True
                    #                 alpha_plus = set(calculate_single_attribute_closure(list(alpha),self.F))
                    #                 if temp_R_set.issubset(alpha_plus):
                    #                     temp = False
                    #                 if temp:
                    #                     test += 1
                    #                     allBCNF = False
                    #                     result.remove(temp_R)
                    #                     result.append(list(temp_R_set.difference(beta_set)))
                    #                     result.append(list(alpha_set.union(beta_set)))
                    #                     temp_done = True
                    #                     break
                # if temp_done:
                #     break
            print(allBCNF,'\t',result,'\t',test)
            if(allBCNF):
                done = True
        return result,step

In [348]:
# R = [i for i in range(1,12)]
# F = [[[1],[2,3,4]],[[8,9],[10]],[[1,5,6,7],[8,9,11]]]
#answer:[{1, 2, 3, 4}, {8, 9, 10}, {1, 5, 6, 7, 8, 9, 11}]
R = [i for i in range(1,4,1)]
F = [[[1],[2]],[[2],[3]]]
#answer：[[2, 3], [1, 2]]
R = [i for i in range(1,5)]
F = [[[1,2],[3]],[[3],[4]],[[4],[1]]]
print(R,F)

[1, 2, 3, 4] [[[1, 2], [3]], [[3], [4]], [[4], [1]]]


In [349]:
nf = NF(R,F)
nf.BCNF()

False 	 [[3, 4], [1, 2, 3]] 	 0
True 	 [[3, 4], [1, 2, 3]] 	 0


([[3, 4], [1, 2, 3]],
 {'root': [1, 2, 3, 4],
  'decompose_dependency': [[3], [4]],
  'left': [3, 4],
  'right': {'root': [1, 2, 3]}})

In [350]:
result = nf.BCNF()

False 	 [[3, 4], [1, 2, 3]] 	 0
True 	 [[3, 4], [1, 2, 3]] 	 0


In [351]:
# for j in result:
#     bc ,vio= nf.check_schemaInBCNF(j)
#     if not bc:
#         print(j,'\t',vio)

TypeError: unhashable type: 'list'

In [ ]:
# for s in nf.F:
#     if (nf.check_NontrivialDependencyCaseViolationOfBCNF(s,result[2])):
#         print(s,calculate_single_attribute_closure(s[0],nf.F))

In [352]:
F_cover = compute_canonical_cover_set(nf.F)
print(F_cover)
nf_cover = NF(R,F_cover)
nf.BCNF()

[[[1, 2], [3]], [[3], [4]], [[4], [1]]]
False 	 [[3, 4], [1, 2, 3]] 	 0
True 	 [[3, 4], [1, 2, 3]] 	 0


([[3, 4], [1, 2, 3]],
 {'root': [1, 2, 3, 4],
  'decompose_dependency': [[3], [4]],
  'left': [3, 4],
  'right': {'root': [1, 2, 3]}})